# EfficientNet for GeoLocation Mapping

In [ ]:
import torch
import torchvision
import timm
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from google.colab import drive
import os

In [ ]:
# Mount uploaded state images from Google Drive
drive.mount('/content/drive/', force_remount=True)
!ls /content/drive/MyDrive/proj/
!ls /content/drive/MyDrive/proj_test/test_data/

## Data Preparation

In [ ]:
# Reshape images to 224x224 and normalize
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

# Define directories for train and test and load them
data_dir_train = "/content/drive/MyDrive/proj"
data_dir_test = "/content/drive/MyDrive/proj_test/test_data"

dataset_train = datasets.ImageFolder(root=data_dir_train, transform=transform, is_valid_file=lambda x: 'info.txt' not in x)
dataset_test = datasets.ImageFolder(root=data_dir_test, transform=transform, is_valid_file=lambda x: 'info.txt' not in x)

In [ ]:
batch_size = 32
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=12)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=False, num_workers=12)

## EfficientNet Model Setup

In [ ]:
# Using b0 -- 16 layers, 224x224
model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=len(dataset_train.classes))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

## Loss Function and Optimizer

In [ ]:
loss_fn = nn.CrossEntropyLoss()
adam_optimizer = optim.Adam(model.parameters(), lr=1e-4)

## Training Loop
### Using 10 epochs

In [ ]:
epochs = 10
print(len(train_loader))
i = 0
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        i += 1
        if i % 10 == 0:
          print(i)
        inputs, labels = inputs.to(device), labels.to(device)
        adam_optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        adam_optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}")

print("Training Complete")

## Evaluation Loop

In [ ]:
model.eval()
correct_top1 = 0
correct_top5 = 0
total = 0
print(len(test_loader))
i = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        if i % 10 == 0:
          print(i)
        i += 1
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        # Top-1 accuracy
        _, predicted = torch.max(outputs, 1)
        correct_top1 += (predicted == labels).sum().item()
        # Top-5 accuracy
        _, top5_pred = torch.topk(outputs, 5, dim=1)
        correct_top5 += sum([labels[i] in top5_pred[i] for i in range(len(labels))])
        total += labels.size(0)

print(f'Top-1 Accuracy: {100 * correct_top1 / total:.2f}%')
print(f'Top-5 Accuracy: {100 * correct_top5 / total:.2f}%')